Process Proposed by Hari Prasad for WML development & deployment. D-I-D. Development Notebook-Implementation Python-Deployment Notebook.

* Create .py model file as per the coding standards.
    * It means we need 3 notebooks. 
        * Development.pynb for initial development of models and runs.
        * Implementation.py which is modified version of Development.pynb with changes as per Watson Coding Standards.
            * Remember that we can seperate data cleansing in a seperate file  and call that in Implementation.Not mandatory.
        * Deployment.pynb is to deploy the Implementation.py to Cloud & Scheck on Scoring.
    * Implementation.py will have the code to connect to cloud storage
        * Train Data Download from data bucket & Cleansing
        * Model Definition
        * Model Compile & Fit
        * Model Save to results bucket

Data Preperation

We will use LSTM to predict the signwave.

In [ ]:
import numpy as np
angle = np.linspace(-np.pi, np.pi, 201)
import matplotlib.pylab as plt
y=np.sin(angle)
print(type(y))
plt.plot(angle,y)

In [ ]:
import pandas as pd
values=list()
for looped in range(0,100):
    for i in range(0,len(y)):
        values.append(y[i]);

In [ ]:
time_step=list()
for i in range(0,len(values)):
        time_step.append(i);

In [ ]:
data = pd.DataFrame(
    {'time_step': time_step,
     'valuea': values,
    })

In [ ]:
plt.plot(data['time_step'][0:800],data['valuea'][0:800])

In [ ]:
series=np.reshape(data['valuea'].values,(np.shape(data['valuea'].values)[0],1))

In [ ]:
np.save("series",series)

We have series repeating every once in 200 cycles.

Preparing data for time series

## Cloud Section to upload the data.

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
import time

In [ ]:
cos_credentials = {
  "apikey": "",
  "cos_hmac_keys": {
    "access_key_id": "",
    "secret_access_key": ""
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/fa381d574e4a999f5fa9dca6b6b90fe7:94201900-eecd-43d2-ad0c-51692cc709b9::",
  "iam_apikey_name": "auto-generated-apikey-14fc88d0-fb45-4b9d-bbf4-ea1eca5df527",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/fa381d574e4a999f5fa9dca6b6b90fe7::serviceid:ServiceId-2eb92714-5e02-48e1-b8cf-d1b31cf204fc",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/fa381d574e4a999f5fa9dca6b6b90fe7:94201900-eecd-43d2-ad0c-51692cc709b9::"
}

api_key = cos_credentials['apikey']
service_instance_id = cos_credentials['resource_instance_id']

In [ ]:
# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

In [ ]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

In [ ]:
# Create a COS resource.
cos = ibm_boto3.resource ('s3',
                         ibm_api_key_id=api_key,
                         ibm_service_instance_id=service_instance_id,
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
# Create buckets.
buckets = ['keras-lstm-data-v5', 'keras-lstm-result-v5']

In [ ]:
# Create buckets.
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [ ]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

In [ ]:
# Upload files to the buckets.
bucket_name = buckets[0]
bucket_obj = cos.Bucket(bucket_name)

In [ ]:
!pip install wget
link = 'https://github.com/meethariprasad/phd/raw/master/series.npy'

In [ ]:
import wget, os

data_dir = 'keras_lstm'

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.isfile(os.path.join(data_dir, os.path.join(link.split('/')[-1]))):
    wget.download(link, out=data_dir)  
        
!ls keras_lstm

In [ ]:
import wget, os

data_dir = 'keras_lstm'

for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename), 'rb') as data: 
        bucket_obj.upload_file(os.path.join(data_dir, filename), filename)
        print('{} is uploaded.'.format(filename))

In [ ]:
# Display a list of buckets.
for obj in bucket_obj.objects.all():
    print('Object key: {}'.format(obj.key))
    print('Object size (kb): {}'.format(obj.size/1024))

### Watson Machine Learning Experiment

In [ ]:
# Import the libraries.
import urllib3, requests, json, base64, time, os
warnings.filterwarnings('ignore')

In [ ]:
wml_credentials = {
  "apikey": "",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/fa381d574e4a999f5fa9dca6b6b90fe7:4277973c-258e-4701-a035-42d54566f2b3::",
  "iam_apikey_name": "auto-generated-apikey-2247a1ae-1b9f-48de-8678-4cb3cc722453",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/fa381d574e4a999f5fa9dca6b6b90fe7::serviceid:ServiceId-0e00fbf7-9a8d-4d4f-b931-7ecd4de1c921",
  "instance_id": "",
  "password": "",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": ""
}

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Display the client version number.
print(client.version)

### Create Training Definition

In [ ]:
model_definition_metadata = {
            client.repository.DefinitionMetaNames.NAME: "keras-lstm-v2",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 implementation_keras_lstm_v2.py"
            }

In [ ]:
filename_keras='implementation_keras_lstm_v2.zip'

In [ ]:
!rm -rf implementation_keras_lstm_v2.zip

if os.path.isfile(filename_keras):
    !ls 'implementation_keras_lstm_v2.zip'
else:
    !wget https://github.com/meethariprasad/phd/raw/master/Watson/Keras-lstm/implementation_keras_lstm_v2.zip
    !ls 'implementation_keras_lstm_v2.zip'

In [ ]:
#Store Definitions in WML Repository
definition_details = client.repository.store_definition(filename_keras, model_definition_metadata)

In [ ]:
definition_url = client.repository.get_definition_url(definition_details)
definition_uid = client.repository.get_definition_uid(definition_details)
print(definition_url)

In [ ]:
client.repository.list_definitions()

In [ ]:
#We need to define Required = Y mandatory to run experiment
client.repository.ExperimentMetaNames.show()

In [ ]:
TRAINING_DATA_REFERENCE = {
                            "connection": {
                                "endpoint_url": service_endpoint,
                                "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                            },
                            "source": {
                                "bucket": buckets[0],
                            },
                            "type": "s3"
}

In [ ]:
TRAINING_RESULTS_REFERENCE = {
                                "connection": {
                                    "endpoint_url": service_endpoint,
                                    "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                    "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                                },
                                "target": {
                                    "bucket": buckets[1],
                                },
                                "type": "s3"
}

In [ ]:
#We need to define Required = Y mandatory to run experiment
client.repository.ExperimentMetaNames.show()

In [ ]:
experiment_metadata = {
            client.repository.ExperimentMetaNames.NAME: "Keras LSTM Experiment",
            #client.repository.ExperimentMetaNames.DESCRIPTION: "KERAS LSTM MODEL",
            #client.repository.ExperimentMetaNames.EVALUATION_METHOD: "multiclass",
            #client.repository.ExperimentMetaNames.EVALUATION_METRICS: ["val_acc"],
            #client.repository.ExperimentMetaNames.EVALUATION_METRICS: [{"name": "val_acc", "maximize": True}],
            client.repository.ExperimentMetaNames.TRAINING_DATA_REFERENCE: TRAINING_DATA_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_RESULTS_REFERENCE: TRAINING_RESULTS_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_REFERENCES: [
                        {
                            "name": "keras_lstm",
                            "training_definition_url": definition_url,
                            "compute_configuration": {"name": "k80x2"}
                            
                        }]
            }

In [ ]:
# Store the experiment, and display the experiments uid.
experiment_details = client.repository.store_experiment(meta_props=experiment_metadata)

experiment_uid = client.repository.get_experiment_uid(experiment_details)
print(experiment_uid)

In [ ]:
client.repository.list_experiments()

In [ ]:
#old_experiment_guid="9607bb06-7e89-41f1-b90c-b574541fb1c5"
#client.repository.delete(old_experiment_guid)

In [ ]:
details = client.repository.get_experiment_details(experiment_uid)
print(details)

Run experiment

In [ ]:
# Start the experiment run.
experiment_run_details = client.experiments.run(experiment_uid, asynchronous=True)

In [ ]:
experiment_run_uid = client.experiments.get_run_uid(experiment_run_details)
print(experiment_run_uid)

In [ ]:
client.experiments.list_runs()

In [ ]:
experiment_run_details = client.experiments.get_run_details(experiment_run_uid)
print(experiment_run_details)

In [ ]:
print(json.dumps(experiment_details, indent=2))

In [ ]:
client.experiments.get_status(experiment_run_uid)['state']

Model

In [ ]:
status = client.experiments.get_status(experiment_run_uid)

In [ ]:
#Got the run ID from Watson Environment.
client.training.get_details('training-WWmHAB5ig')

In [ ]:
model_uid='training-WWmHAB5ig'
saved_model_details = client.repository.store_model(model_uid, {'name': 'Keras LSTM model'})

In [ ]:
model_guid = client.repository.get_model_uid(saved_model_details)
print("Saved model guid: " + model_guid)

In [ ]:
client.repository.list_models()

In [ ]:
deployment_details = client.deployments.create(model_guid, name="Keras LSTM deployment")

In [ ]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

Scoring

In [ ]:
import numpy as np
angle = np.linspace(-np.pi, np.pi, 201)
#import matplotlib.pylab as plt
y=np.sin(angle)
#print(type(y))
#plt.plot(angle,y)


# In[4]:


import pandas as pd
values=list()
for looped in range(0,100):
    for i in range(0,len(y)):
        values.append(y[i]);


# In[5]:


time_step=list()
for i in range(0,len(values)):
        time_step.append(i);


# In[6]:


data = pd.DataFrame(
    {'time_step': time_step,
     'valuea': values,
    })


# In[7]:


#plt.plot(data['time_step'][0:800],data['valuea'][0:800])


# In[8]:


series=np.reshape(data['valuea'].values,(np.shape(data['valuea'].values)[0],1))


# We have series repeating every once in 200 cycles.
# 
# Preparing data for time series

# In[9]:


#Normalize Values
from sklearn.preprocessing import MinMaxScaler
#Sin wave b/w -1 to 1
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler_fit=scaler.fit(series)
scaled = scaler_fit.transform(series)
series = pd.DataFrame(scaled)


# During new data incoming, remember to use scaler used above to scale the data.

# In[10]:


window_size = 201

series_s = series.copy()
for i in range(window_size):
    series = pd.concat([series, series_s.shift(-(i+1))], axis = 1)
    
series.dropna(axis=0, inplace=True)


# In[11]:


#series.shape


# In[12]:


nrow = round(0.8*series.shape[0])


# In[13]:


train = series.iloc[:nrow, :]
test = series.iloc[nrow:,:]


# In[14]:


#test.to_csv("time_series_train.csv")
#test.to_csv("time_series_test.csv")


# In[15]:


from sklearn.utils import shuffle
train = shuffle(train)


# In[16]:


train_X = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
test_X = test.iloc[:,:-1]
test_y = test.iloc[:,-1]


# In[17]:


train_X = train_X.values
train_y = train_y.values
test_X = test_X.values
test_y = test_y.values


# In[18]:


#print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)


# Reshaping the data to match with LSTM

# In[19]:


train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],1)
test_X = test_X.reshape(test_X.shape[0],test_X.shape[1],1)

In [ ]:
print(test_X[0:202].shape)
payload_data=test_X[0:202]

In [ ]:
# Example: scoring_payload = {'fields': ['GENDER','AGE','MARITAL_STATUS','PROFESSION'], 'values': [['M',23,'Single','Student'],['M',55,'Single','Executive']]}
 

In [ ]:
scoring_data = {'values': payload_data.tolist()}

In [ ]:
predictions = client.deployments.score(scoring_url, scoring_data)
print("Scoring result: " + str(predictions))

In [ ]:
test_y[0:202].shape

In [ ]:
predictions

In [ ]:
prediction_frame=pd.DataFrame(columns=(predictions['fields']))
prediction_frame

In [ ]:
for i in range(0,len(predictions['values'])):
    if i==0:
        prediction_data=np.transpose(np.array(predictions['values'][i]))
    if i>0:
        temp=np.transpose(np.array(predictions['values'][i]))
        prediction_data=np.append(pred_data,temp,axis=0)

In [ ]:
prediction_dataframe=pd.DataFrame(prediction_data,columns=(predictions['fields']))
prediction_dataframe.iloc[:,0:1].shape

In [ ]:
actual=test_y[0:203]
actuals_dataframe=pd.DataFrame(actual,columns=['actual'])
actuals_dataframe.shape

In [ ]:
predict_actual_df=pd.concat([prediction_dataframe.iloc[:,0:1], actuals_dataframe], axis=1)
predict_actual_df.shape

In [ ]:
#We have scaled the data inside -1 to 1 to again -1 to 1. May not make much sense in this example, but for other real data we might want to transform and re-transform.
scaler_fit.inverse_transform(predict_actual_df).shape

In [ ]:
predict_actual_df.columns

We will plot and see the prediction and actual. We can see that it is doing well on test data.

In [ ]:
import plotly as pltly
import plotly.tools as tls

In [ ]:
import matplotlib.pyplot as plt
mat_plt_plt=plt.figure()
plt.plot(predict_actual_df['prediction'][0:200])
plt.plot(predict_actual_df['actual'][0:200])

How about zooming in and see how close the predicted and actual plots are?

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
plotly_fig = tls.mpl_to_plotly(mat_plt_plt)
iplot(plotly_fig, filename='myplot')

Statistically Checking MSE,RMSE,R Squared on test data.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
meanSquaredError=mean_squared_error(predict_actual_df['actual'], predict_actual_df['prediction'])
print("MSE:", meanSquaredError)
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)
from sklearn.metrics import r2_score
print("R Squared:",r2_score(predict_actual_df['actual'], predict_actual_df['prediction']) )

## Next steps

Given 201 time steps, the model is predicting 202 value.

Some realtime steps are missing though.

Model is trained once and used without updates. 

There is a possibility that as and when future data arrives, the accuracy of the model reduces beond acceptable level.

One way to ensure model quality is within accceptable level levels is, predict for next value, then when the acctual next data arrives check the difference between actual/predicted and check the model quality. If the model tolerance below the acceptable limit, then retain the model based on new actual values and check if the model quality is increased. If it is not with tolerance limit, set up a alert to data scientist for further research in to model quality. Stakeholders also need to be informed about tolerance level dropped, ensuring they don't infer based on below threshold model. Best way is to put the model quality in dashboard accessible for both stakeholders and data scientists.

The answer for above can be explored further in continuous learning.




